#Using Regular Expressions to Organize Data

In [161]:
#Library 
import re

In [162]:
#opening data files to read

#tells how to read file by splitting every time enter is typed (\n)
readings = open('sabrina.dat', 'r').read().split('\n')
    #loops through all lines from the file
for line in readings:
    print(line)


Header information
 
Source: IC443
RA:    6 16 43.60        DEC:  22 32 37.00   (J2000)
OBS. DATE: 20140913          Time of record:   1/  0 37 30.0
No. channels:   512           First channel plotted     1
Bw (kHz):      0.610
Antenna #  2     name: RT1
Antenna #  4     name: RT3
Rest freq. (MHz) :    1720.5300      Av. weight :       0.0143
 
DATA follow with format: (1X,I5,2X,I3,2X,A5,2X,F12.4,2(2X,G15.6),2X,F8.3)
 
Channel  IF  Polar     Frequency     Velocity         Ampl(Jy)      Phase
     1    1  RR          1720.8099      0.00000      FLAGGED
     2    1  RR          1720.8093      0.00000         0.258678E-03     6.040
     3    1  RR          1720.8087      0.00000         0.463556E-03   -95.605
     4    1  RR          1720.8081      0.00000         0.522407E-03  -106.389
     5    1  RR          1720.8075      0.00000         0.422755E-03   -20.039
     6    1  RR          1720.8069      0.00000         0.863785E-03  -122.188
     7    1  RR          1720.8063      0.00000

#Problem 
This data needs to be reorganized not in terms of it's data in the columns but each chunk of data is randomly thrown in. In the headings of these chunks it reads 'Time of record:' fallowed by numbers like '1/ 0 37 30.0'. What these numbers are is the 
day/ hr. minute timed_ave.
of that chunk of data. All the data was collected on the same day and in 30min time averages so those will not change. However, the hr and minute vary between data chunks. Therefore my task is to organize this so that the chunks of data are in chronological order by these minute and hr, using regular expressions.For this assignment I will describe what my over all goal of reorganizing is but I will focus on the portion that is just grabbing these hour-minute numbers.

#Steps to Solution
1)Identify how to capture just those numbers
    -use positive and negative test groups for reasurance
    
2)Create a function containg a loop that will print the captured values 
    -extract numbers captured
    -print only numbers and not none results
    
3)Describe further steps to organize whole data set
    -create loop for chronological ordering
    -make numbers reference chunks of data
    -make function that uses these loops and prints 


#Step 1

In [163]:
#Identify how to capture those numbers 
#looking at just header first
#record:   1/  0 37 30.0
match = re.search('\s([\s][0-9]|[0-9][0-9])\s([\s][0-9]|[0-9][0-9])\s', 'Time of record:   1/  0 37 30.0')

In [164]:
#successfully captured hour and minute
print(match.group(1),match.group(2))

 0 37


In [165]:
#Check to make sure that works for other times
match = re.search('\s([\s][0-9]|[0-9][0-9])\s([\s][0-9]|[0-9][0-9])\s', 'Time of record:   1/  1  8 30.0')
print(match.group(1),match.group(2))

 1  8


In [166]:
#expand capture code so it only works for only'Time of record'
#test with other similar number format from heading

match = re.search('\s([\s][0-9]|[0-9][0-9])\s([\s][0-9]|[0-9][0-9])\s30.0', 'DEC:  22 32 37.00   (J2000)')

#I know every time record is ended with 30.0 so I used that as a reference

In [167]:
#Test should not return results
print(match.group(1),match.group(2))

AttributeError: 'NoneType' object has no attribute 'group'

In [168]:
#Test should return 1  8
match = re.search('\s([\s][0-9]|[0-9][0-9])\s([\s][0-9]|[0-9][0-9])\s30.0','Time of record:   1/  1  8 30.0')
print(match.group(1),match.group(2))

 1  8


#Step 2

In [169]:
#Create function to grab only hour minute
#assign variable to capture pattern
cp='\s([\s][0-9]|[0-9][0-9])\s([\s][0-9]|[0-9][0-9])\s30.0'

def show_groups(pattern, text):
    
    m= re.search(pattern, text)

    #Create loop to print matches
    if m is None:
        print('No match')
        return
   
    for i in range(1, 1+ len(m.groups())):
          print(m.group(1),m.group(2))


In [170]:
#Test function
show_groups(cp, 'Time of record:   1/  1  8 30.0')

 1  8
 1  8


In [171]:
#Apply similar function method to print matches from whole data set 
def get_time(record):
    '''Return (hour,minute) as string'''
    
    #try re-search() for times 
    p=re.search(cp, record)
    
    #loop for research to return above pattern
    if p:
        return p.group(1), p.group(2)

#add loop so that no-matches aren't printed
for r in readings:
    #assign variable to function
    time=get_time(r)
    if time:
        print(get_time(r))

(' 0', '37')
(' 1', ' 8')
(' 1', '39')
(' 2', '10')
(' 2', '41')
(' 3', '12')
(' 3', '43')
(' 5', '16')
(' 5', '47')
(' 6', '18')
(' 6', '49')
(' 7', '20')
(' 7', '51')
(' 8', '22')
(' 8', '53')
(' 9', '24')
(' 9', '55')
('11', '28')
('11', '59')


#Step 3

To continue the organization from here I would need to have the numbers my function grabbed still reference the chunk of data there from. I would then include this in a new function with a loop that would read these numbers and put them, along with their specific chunk of data, in chronological order. 

#Discussion of results
Although I was not able to completely re-organize my data chronologically, the list of numbers I captured do tell me interest information that is useful when looking through my data. What they show me is that there is not a chunk of data present for every hour and every minute. As the 30minute time average would suggest the data chunks refence minutes and hours that differ by a 30minute time period and so they are already in chronological order. With out doing this though it would have been much harder for me to scroll through all my data to look at each heading to see when the times were. I also see here that there is a 1minute time delay in starting the next 30minute time ave. collection of data. With just minor adjustments this function will also be useful when I have multiple days of data to go through.